In [24]:
import os, sys

In [25]:
model_id = 'ContinuousSBM_FFResUNet_bottleneck_32_firstc_10_phi_beta_cosine_betamax_2.0_betamin_0.01' #'ContinuousSBM_bottleneck_32_firstc_10_phi_beta_cosine_betamax_2.0_betamin_0.01' #'ContinuousSBM_ContinuousVPSDE_I_BPROJ_bottleneck_32_firstc_10_phi_beta_cosine_betamax_0.5_betamin_0.001'
output_dir ='/mnt/home/dheurtel/ceph/04_inference/sbc/'+model_id
num_samples_per_chain = 600
num_sims_per_gpu = 256
num_chains_per_sim = 2
nb_gpus = 4
noise_level_min = 0.05
noise_level_max = 0.5

In [26]:
script_template = \
"""#!/bin/bash
#SBATCH -J sbcv2_SigmaMN_{{count}}
#SBATCH --gpus=1
#SBATCH --cpus-per-gpu=16
#SBATCH --time=48:00:00
#SBATCH --partition=gpu
#SBATCH --constraint='a100-80gb|h100'
#SBATCH -o jobs/sbcv2_SigmaMN_{{count}}.log

source ~/.bashrc
source /mnt/home/dheurtel/venv/genv_DL/bin/activate
python sbc_with_MN_and_timev2.py --model_id {{model_id}} --num_target={{num_sims_per_gpu}} --num_chain_per_target={{num_chains_per_sim}} --num_sample={{num_samples_per_chain}} --save_path={{save_path}} --noise_level_min={{noise_level_min}} --noise_level_max={{noise_level_max}} 
"""

In [27]:
keys_substitutes = {"{{model_id}}": model_id,
                    "{{num_sims_per_gpu}}": num_sims_per_gpu,
                    "{{num_samples_per_chain}}": num_samples_per_chain,
                    "{{num_chains_per_sim}}": num_chains_per_sim,
                    "{{save_path}}": os.path.join(output_dir, "tmp.pt"),
                    "{{count}}": 0,
                    "{{noise_level_min}}": noise_level_min,
                    "{{noise_level_max}}": noise_level_max}

In [28]:
for gpu_id in range(nb_gpus):
    keys_substitutes["{{count}}"] = gpu_id
    keys_substitutes["{{save_path}}"] = os.path.join(output_dir, f"sbcv2_sims_SigmaMN_{gpu_id}.pt")
    script = script_template
    for key, value in keys_substitutes.items():
        script = script.replace(key, str(value))
    with open(f'./jobs/sbcv2_sims_SigmaMN_FF_{gpu_id}.sh', 'w') as f:
        f.write(script)
    os.system(f'sbatch ./jobs/sbcv2_sims_SigmaMN_FF_{gpu_id}.sh')

Submitted batch job 3162282
Submitted batch job 3162283
Submitted batch job 3162284
Submitted batch job 3162285
